# DuckDB in python
Notebook that follows the [comprehensive guide](https://medium.com/@anshubantra/using-duckdb-in-python-a-comprehensive-guide-d14bc0b06546) pusblished in medium platform at _September 2024_.

In [1]:
import duckdb as dd
import pandas as pd
from pathlib import Path

repo_path = Path().resolve().parent # local repository path

Create a `DuckDB` database from scratch
 * database `in-memory` assuming that data is stored in RAM memory
 * database "persistent" assuming that data is stored in a file, making persistent between sessions

In [4]:
# in-memory DuckDB connection
con_ram = dd.connect(':memory:')

# persistent DuckDB connection
con_per = dd.connect(repo_path / 'data/my_database.db')

See the database schema:

In [5]:
print('in-memory database\n')
display(con_ram.sql('show all tables'))
print('persistent database\n')
display(con_per.sql('show all tables'))

in-memory database



┌──────────┬─────────┬─────────┬──────────────┬──────────────┬───────────┐
│ database │ schema  │  name   │ column_names │ column_types │ temporary │
│ varchar  │ varchar │ varchar │  varchar[]   │  varchar[]   │  boolean  │
├──────────┴─────────┴─────────┴──────────────┴──────────────┴───────────┤
│                                 0 rows                                 │
└────────────────────────────────────────────────────────────────────────┘

persistent database



┌─────────────┬─────────┬───────────┬──────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name    │                       column_names                       │                 column_types                  │ temporary │
│   varchar   │ varchar │  varchar  │                        varchar[]                         │                   varchar[]                   │  boolean  │
├─────────────┼─────────┼───────────┼──────────────────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────┤
│ my_database │ main    │ countries │ [country, code, region, sub_region, intermediate_region] │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR] │ false     │
│ my_database │ main    │ employees │ [id, name, age, salary]                                  │ [INTEGER, VARCHAR, INTEGER, DOUBLE]           │ false     │
└─────────────┴─────────┴───────────┴─────────────────────

## Basic usage of DuckDB
You can start running SQL queries directly with DuckDB in python, here is basic example:

In [4]:
# Running a basic SQL query
result = dd.sql("SELECT 'DuckDB_is_cool' AS answer")
print(type(result))
print(result.fetchall())
print(result)

<class 'duckdb.duckdb.DuckDBPyRelation'>
[('DuckDB_is_cool',)]
┌────────────────┐
│     answer     │
│    varchar     │
├────────────────┤
│ DuckDB_is_cool │
└────────────────┘



In [36]:
# Create a relation from a SQL query
rel = dd.sql("SELECT * FROM range(10_00) AS tbl(ID)")
# Display the relation
rel.show()

┌────────────┐
│     ID     │
│   int64    │
├────────────┤
│          0 │
│          1 │
│          2 │
│          3 │
│          4 │
│          5 │
│          6 │
│          7 │
│          8 │
│          9 │
│          · │
│          · │
│          · │
│        990 │
│        991 │
│        992 │
│        993 │
│        994 │
│        995 │
│        996 │
│        997 │
│        998 │
│        999 │
├────────────┤
│ 1000 rows  │
│ (20 shown) │
└────────────┘



## DuckDB for SQL queries and data ingestion
DuckDB supports any SQL syntax and you can definitely use as "interface" with SQL without the trouble if using tools such as `SQL mgnt studio` or `DBeaver`. You can use DuckDB instead and keep working on the python world.

Let us use the `con_per` to use the `my_database.db` that this notebook creates, and insert manually a table:

In [37]:
# Create a table and its schema
con_per.execute('''
CREATE OR REPLACE TABLE countries (
    country VARCHAR,
    code VARCHAR,
    region VARCHAR,
    sub_region VARCHAR,
    intermediate_region VARCHAR
);
''')

# Insert some data into 'countries' table
con_per.execute('''
INSERT INTO countries VALUES
('Australia', 'AUS', 'Oceania', 'Australia and New Zealand', ''),
('India', 'IND', 'Asia', 'Southern Asia', '');
''')

con_per.sql('SHOW ALL TABLES')

┌─────────────┬─────────┬───────────┬──────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name    │                       column_names                       │                 column_types                  │ temporary │
│   varchar   │ varchar │  varchar  │                        varchar[]                         │                   varchar[]                   │  boolean  │
├─────────────┼─────────┼───────────┼──────────────────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────┤
│ my_database │ main    │ countries │ [country, code, region, sub_region, intermediate_region] │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR] │ false     │
│ my_database │ main    │ employees │ [id, name, age, salary]                                  │ [INTEGER, VARCHAR, INTEGER, DOUBLE]           │ false     │
└─────────────┴─────────┴───────────┴─────────────────────

In [6]:
# Show the table
con_per.sql('select * from countries')

┌────────────────────────────────────┬─────────┬──────────┬─────────────────────────────────┬─────────────────────┐
│              country               │  code   │  region  │           sub_region            │ intermediate_region │
│              varchar               │ varchar │ varchar  │             varchar             │       varchar       │
├────────────────────────────────────┼─────────┼──────────┼─────────────────────────────────┼─────────────────────┤
│ Afghanistan                        │ AFG     │ Asia     │ Southern Asia                   │ NULL                │
│ Åland Islands                      │ ALA     │ Europe   │ Northern Europe                 │ NULL                │
│ Albania                            │ ALB     │ Europe   │ Southern Europe                 │ NULL                │
│ Algeria                            │ DZA     │ Africa   │ Northern Africa                 │ NULL                │
│ American Samoa                     │ ASM     │ Oceania  │ Polynesia   

Now, you can see that we successfully create the table `countries` on the `domain = main` (default) while before `my_database.db` was empty.

---

Let us create the $2^{nd}$ table for `employees` so we can map employees in each country

In [39]:
# Create second table
con_per.execute('''
CREATE OR REPLACE TABLE employees (
    id INTEGER,
    name VARCHAR,
    age INTEGER,
    salary DOUBLE
);
''')

# Insert some data in second table
con_per.execute('''
INSERT INTO employees VALUES
(1, 'Person 1', 30, 70000),
(2, 'Person 2', 25, 55000),
(3, 'Person 3', 35, 80000);
''')


# Result of showing tables after creating the second table
con_per.sql('SHOW ALL TABLES')

┌─────────────┬─────────┬───────────┬──────────────────────────────────────────────────────────┬───────────────────────────────────────────────┬───────────┐
│  database   │ schema  │   name    │                       column_names                       │                 column_types                  │ temporary │
│   varchar   │ varchar │  varchar  │                        varchar[]                         │                   varchar[]                   │  boolean  │
├─────────────┼─────────┼───────────┼──────────────────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────┤
│ my_database │ main    │ countries │ [country, code, region, sub_region, intermediate_region] │ [VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR] │ false     │
│ my_database │ main    │ employees │ [id, name, age, salary]                                  │ [INTEGER, VARCHAR, INTEGER, DOUBLE]           │ false     │
└─────────────┴─────────┴───────────┴─────────────────────

In [7]:
con_per.sql('select * from employees').show()

┌───────┬──────────┬───────┬──────────┐
│  id   │   name   │  age  │  salary  │
│ int32 │ varchar  │ int32 │  double  │
├───────┼──────────┼───────┼──────────┤
│     1 │ Person 1 │    30 │  70000.0 │
│     2 │ Person 2 │    25 │  55000.0 │
│     3 │ Person 3 │    35 │  80000.0 │
│     4 │ Person 4 │    45 │ 100000.0 │
│     5 │ Person 5 │    58 │  85000.0 │
│     6 │ Person 6 │    61 │ 175000.0 │
│     7 │ Person 7 │    35 │  90000.0 │
│     8 │ Person 8 │    26 │  65000.0 │
│     9 │ Person 9 │    31 │  75000.0 │
└───────┴──────────┴───────┴──────────┘



## Real example with DuckDB
Example above is basic and for learning reasons mostly. On the other hand, real examples where we handle with _large datasets_ we cannot create from scratch the database. Therefore, we resort ourselves to data stored in files from various sources, `.csv`, `.json`, `.parquet`, etc. DuckDB is fully compatible with ingesting data from various source files, therefore we will ingest the table `countries` from a `.parquet` file instead.

First, we need to delete the existing table created:

In [7]:
con_per.sql('delete from countries')
con_per.sql('select * from countries').show()

┌─────────┬─────────┬─────────┬────────────┬─────────────────────┐
│ country │  code   │ region  │ sub_region │ intermediate_region │
│ varchar │ varchar │ varchar │  varchar   │       varchar       │
├─────────┴─────────┴─────────┴────────────┴─────────────────────┤
│                             0 rows                             │
└────────────────────────────────────────────────────────────────┘



Now, let's ingest the table `countries` from the `.parquet` stored in `data/` folder.

In [ ]:
parquet_file = repo_path / 'data/countries.parquet' # parquet file location

# Ingest parquet file as 'countries' table
con_per.sql(f'''
    insert into countries (country, code, region, sub_region, intermediate_region)
    (select * from read_parquet('{parquet_file}'))
''')
# Example if we had to ingest a .csv file
# con_per.sql(f'''
#     INSERT INTO countries (country, code, region, sub_region, intermediate_region)
#     SELECT * FROM read_csv_auto('{csv_file}')
# ''')

# Show the ingested table
con_per.sql('select * from countries limit 5').show()

┌────────────────┬─────────┬─────────┬─────────────────┬─────────────────────┐
│    country     │  code   │ region  │   sub_region    │ intermediate_region │
│    varchar     │ varchar │ varchar │     varchar     │       varchar       │
├────────────────┼─────────┼─────────┼─────────────────┼─────────────────────┤
│ Afghanistan    │ AFG     │ Asia    │ Southern Asia   │ NULL                │
│ Åland Islands  │ ALA     │ Europe  │ Northern Europe │ NULL                │
│ Albania        │ ALB     │ Europe  │ Southern Europe │ NULL                │
│ Algeria        │ DZA     │ Africa  │ Northern Africa │ NULL                │
│ American Samoa │ ASM     │ Oceania │ Polynesia       │ NULL                │
└────────────────┴─────────┴─────────┴─────────────────┴─────────────────────┘



Let us do some basic SQL query in the new table, it will work as if we were in `SQL mngt studio` SW.

In [8]:
# Parameters for dynamic query
region = 'Oceania'
sub_region = 'Polynesia'

# SQL query
con_per.sql(f'''
    select * from countries
        where region = '{region}'
            and sub_region = '{sub_region}'
''')

┌───────────────────┬─────────┬─────────┬────────────┬─────────────────────┐
│      country      │  code   │ region  │ sub_region │ intermediate_region │
│      varchar      │ varchar │ varchar │  varchar   │       varchar       │
├───────────────────┼─────────┼─────────┼────────────┼─────────────────────┤
│ American Samoa    │ ASM     │ Oceania │ Polynesia  │ NULL                │
│ Cook Islands      │ COK     │ Oceania │ Polynesia  │ NULL                │
│ French Polynesia  │ PYF     │ Oceania │ Polynesia  │ NULL                │
│ Niue              │ NIU     │ Oceania │ Polynesia  │ NULL                │
│ Pitcairn          │ PCN     │ Oceania │ Polynesia  │ NULL                │
│ Samoa             │ WSM     │ Oceania │ Polynesia  │ NULL                │
│ Tokelau           │ TKL     │ Oceania │ Polynesia  │ NULL                │
│ Tonga             │ TON     │ Oceania │ Polynesia  │ NULL                │
│ Tuvalu            │ TUV     │ Oceania │ Polynesia  │ NULL                │

#### Remark
The `.sql` in DuckDB actually reads from these files into an in-memory DuckDB relation (table) and query them directly, to explore and work with data.

This is known as DuckDB **relation objects** and can look into all data in this object.

In [9]:
relation = con_per.sql(f'''
    select * from countries
        where region = '{region}'
            and sub_region = '{sub_region}'
''')

relation.fetchall()

[('American Samoa', 'ASM', 'Oceania', 'Polynesia', None),
 ('Cook Islands', 'COK', 'Oceania', 'Polynesia', None),
 ('French Polynesia', 'PYF', 'Oceania', 'Polynesia', None),
 ('Niue', 'NIU', 'Oceania', 'Polynesia', None),
 ('Pitcairn', 'PCN', 'Oceania', 'Polynesia', None),
 ('Samoa', 'WSM', 'Oceania', 'Polynesia', None),
 ('Tokelau', 'TKL', 'Oceania', 'Polynesia', None),
 ('Tonga', 'TON', 'Oceania', 'Polynesia', None),
 ('Tuvalu', 'TUV', 'Oceania', 'Polynesia', None),
 ('Wallis and Futuna', 'WLF', 'Oceania', 'Polynesia', None)]

 * This cmd `fetchall()` gets all entries from the `relation` object as a list of tuples.
 * We can also convert the `relation` table into a `DataFrame`

In [10]:
relation.to_df()

,country,code,region,sub_region,intermediate_region
0,American Samoa,ASM,Oceania,Polynesia,None
1,Cook Islands,COK,Oceania,Polynesia,None
2,French Polynesia,PYF,Oceania,Polynesia,None
3,Niue,NIU,Oceania,Polynesia,None
4,Pitcairn,PCN,Oceania,Polynesia,None
5,Samoa,WSM,Oceania,Polynesia,None
6,Tokelau,TKL,Oceania,Polynesia,None
7,Tonga,TON,Oceania,Polynesia,None
8,Tuvalu,TUV,Oceania,Polynesia,None
9,Wallis and Futuna,WLF,Oceania,Polynesia,None


DuckDB has a set of cmd startwith `to_*` that allows you to convert `relation` tables into other formats for data manipulation too (e.g. `DataFrame`, `CSV`, `arrow_table`).

---

* Example of ingesting `DataFrame` into DuckDB database

In [31]:
# Data stored as a DataFrame
df = pd.DataFrame(
    data={
        'id': [4, 5, 6],
        'name': ['Person 4', 'Person 5', 'Person 6'],
        'age': [45, 58, 61],
        'salary': [100000, 85000, 175000],
    }
)

# Ingest as part of the 'employee' table
con_per.sql('''
    insert into employees (id, name, age, salary)
    select * from df
''')

In [32]:
# Show updated table
con_per.sql('select * from employees')

┌───────┬──────────┬───────┬──────────┐
│  id   │   name   │  age  │  salary  │
│ int32 │ varchar  │ int32 │  double  │
├───────┼──────────┼───────┼──────────┤
│     1 │ Person 1 │    30 │  70000.0 │
│     2 │ Person 2 │    25 │  55000.0 │
│     3 │ Person 3 │    35 │  80000.0 │
│     4 │ Person 4 │    45 │ 100000.0 │
│     5 │ Person 5 │    58 │  85000.0 │
│     6 │ Person 6 │    61 │ 175000.0 │
└───────┴──────────┴───────┴──────────┘

Example of inserting the table in a dynamic and generic way:

In [ ]:
# Example DataFrame
df = pd.DataFrame({
    'id': [7, 8, 9],
    'name': ['Person 7', 'Person 8', 'Person 9'],
    'age': [35, 26, 31],
    'salary': [90000, 65000, 75000],
})

# Dynamic target table and view name
table_name = 'employees'
view_name = 'temp_df_view'

# Register DataFrame as a view
con_per.register(view_name, df)

# Dynamically extract column names from the DataFrame
cols = ', '.join(df.columns)

# Build and execute the SQL INSERT statement
con_per.sql(f'''
    insert into {table_name} ({cols})
    select * from {view_name}
''')

# Optional: Unregister the view
con_per.unregister(view_name)

In [34]:
# Show updated table
con_per.sql(f'select * from {table_name}')

┌───────┬──────────┬───────┬──────────┐
│  id   │   name   │  age  │  salary  │
│ int32 │ varchar  │ int32 │  double  │
├───────┼──────────┼───────┼──────────┤
│     1 │ Person 1 │    30 │  70000.0 │
│     2 │ Person 2 │    25 │  55000.0 │
│     3 │ Person 3 │    35 │  80000.0 │
│     4 │ Person 4 │    45 │ 100000.0 │
│     5 │ Person 5 │    58 │  85000.0 │
│     6 │ Person 6 │    61 │ 175000.0 │
│     7 │ Person 7 │    35 │  90000.0 │
│     8 │ Person 8 │    26 │  65000.0 │
│     9 │ Person 9 │    31 │  75000.0 │
└───────┴──────────┴───────┴──────────┘

## DuckDB for analytics work
This module is optimized for SQL queries so-called analytical queries:
1. _Windows functions:_ perform window operations (i.e. sum, average, other stats) over big datasets
1. _Group by:_ Aggregation operations involving _groupby_
1. _Parallel execution:_ DuckDB takes care of parallelizing operations for you under the hood

DuckDB uses a vectorized execution that helps it to speed up query processing and this can be handy to data analytics work. Additionally, this module directly ingestes compressed data formats such as `.parquet`, reducing the need the _pre-requisite_ for a data decompression process/tool.

* Example of _group by_ operation where the objective is to know the number of _unique_ countries each `region` has:

In [ ]:
query = '''
    select region, count(distinct country) as country_counts
        from countries
        group by region
        order by country_counts desc
'''

con_per.sql(query).show() # Execute the query to agg number of unique countries per region

┌──────────┬────────────────┐
│  region  │ country_counts │
│ varchar  │     int64      │
├──────────┼────────────────┤
│ Africa   │             60 │
│ Americas │             57 │
│ Europe   │             51 │
│ Asia     │             51 │
│ Oceania  │             29 │
│ NULL     │              1 │
└──────────┴────────────────┘



Second example is to get the `employees` above the average salary. This involves:
1. Calculate the `average` salary on the `employees` table
1. Select _only_ the employees with `salary > avg_salary`

In [18]:
query = '''
    -- Calculate the average salary -- round(avg(salary), 2); you could round the average if you want to
    with threshold as (
        select avg(salary) as avg_salary from employees
    )
    -- Select only the employees with salary above the threshold
    select * from employees
        where salary > (select avg_salary from threshold)
        order by salary desc
'''

con_per.sql(query).show() # Execute query

┌───────┬──────────┬───────┬──────────┐
│  id   │   name   │  age  │  salary  │
│ int32 │ varchar  │ int32 │  double  │
├───────┼──────────┼───────┼──────────┤
│     6 │ Person 6 │    61 │ 175000.0 │
│     4 │ Person 4 │    45 │ 100000.0 │
│     7 │ Person 7 │    35 │  90000.0 │
└───────┴──────────┴───────┴──────────┘



In [19]:
con_per.sql(query).to_df() # Convert executed query to DataFrame

,id,name,age,salary
0,6,Person 6,61,175000.0
1,4,Person 4,45,100000.0
2,7,Person 7,35,90000.0


## Closing the connection on DuckDB
_Once you’re done with your queries, always remember to close the DuckDB connection._

In [20]:
con_ram.close()
con_per.close()

# Closing arguments
Why using DuckDB instead of a more generic module as `pandas`? This comes to performance when dealing with big datasets `in-memory`. From my own experience, `pandas` is tailored for `in-memory` data manipulation and that's why it became such a popular tool for all around data analytics projects. When dealing with big dataset, I had to switch to other tools, for example `pyspark` and `dask`, in order to take full advantage of the _distributed computing_. DuckDB falls into the same category with a better interface between SQL and `pandas`, which makes it easier to adopt it in my opinion.

As great advantages for me with DuckDB:
1. Easy interface with SQL world and a nice module to leverage all my SQL knowledge while still living in the python/DataFrame world. This helps me in data analytics work.
1. Smooth interface with _compressed_ data formats as `.parquet` where you can ingest the dataset without having to leverage the extra pain that comes with _compressed_ data formats.
1. User friendly module for someone that wants to still use the SQL for data manipulation while keep working in python and DataFrame

## Conclusion from the comprehensive guide
I copy/paste here the conclusion of the [guide](https://medium.com/@anshubantra/using-duckdb-in-python-a-comprehensive-guide-d14bc0b06546) and give full credit to the guide's author:

> "Speed Comparison: DuckDB vs Pandas
> * DuckDB and Pandas are both popular tools for data manipulation in Python, but they have different strengths. Pandas is a general-purpose data manipulation library optimized for in-memory operations, whereas DuckDB is designed specifically for high-performance, analytical queries on large datasets using SQL. DuckDB’s columnar storage and query optimization techniques make it significantly faster than Pandas for complex and large-scale analytical queries.

> Why is DuckDB Faster for Analytical Queries?
> * Columnar Storage: DuckDB stores data in a columnar format, which is more efficient for analytical queries (like filtering and aggregations). Pandas store data in a row-major format, which is better for general-purpose operations but can be slower for these specific tasks.
> * Query Optimization: DuckDB uses query optimizations like predicate pushdown, efficient joins, and parallel query execution, which are typically missing or less efficient in Pandas.
> * Parallel Execution: DuckDB can automatically parallelize complex queries, leveraging multiple cores in modern CPUs, while Pandas processes data mostly single-threaded by default.
> * On-Disk Storage: DuckDB efficiently handles datasets that don’t fit into memory by using on-disk storage formats like Parquet, while Pandas requires that all data fit into memory."